# An Introduction to Reed-Solomon Codes

## Part 1: Polynomials and Finite Fields  
### Polynomials and Finite Fields 

The magic behind Reed-Solomon codes lies in the algebra of **polynomials over finite fields**. Before we can build the codes, we need to build our tools. This section covers the essentials of creating and working with these special mathematical objects.

### A Quick Recap of Polynomials

Let's quickly refresh some basic definitions. We'll be working with a **finite field**, which you can think of as a set of numbers where addition, subtraction, multiplication, and division are all well-defined. The simplest example is $F_p = \{0, 1, \dots, p - 1\}$ where $p$ is a prime number, and all operations are done modulo $p$.

- A **polynomial** $F(X)$ over a field $F_q$ is an expression of the form  
  $$F(X) = f_d X^d + \dots + f_1 X + f_0$$,  
  where the coefficients $f_i$ are all elements of $F_q$.

- The **degree** of the polynomial, $\text{deg}(F)$, is the highest power of $X$ with a non-zero coefficient.

- **Evaluation** means plugging in a value $\alpha \in F_q$ for $X$ to get a result $F(\alpha) \in F_q$.  
  An $\alpha$ is a **root** of $F(X)$ if $F(\alpha) = 0$.

- **Degree Mantra** :a non-zero polynomial of degree $t$ over a field can have at most $t$ distinct roots.

### Irreducibility and Field Extensions

An **irreducible polynomial** is a polynomial that cannot be factored into two non-constant polynomials of smaller degree.

For example, over $\mathbb{F}_2$, the polynomial $X^2 + X + 1$ is irreducible because its only possible factors are $X$ and $X + 1$, and neither divides it. However, $X^2 + 1$ is reducible over $\mathbb{F}_2$ since $X^2 + 1 = (X + 1)(X + 1)$.

The most powerful application of irreducible polynomials is creating **field extensions**. If we have a prime field $\mathbb{F}_p$ and an irreducible polynomial $E(X)$ of degree $s$ over that field, we can construct a new, larger field:

$$
\mathbb{F}_{p^s} \triangleq \mathbb{F}_p[X]/E(X)
$$

This new field contains all polynomials over $\mathbb{F}_p$ with degree less than $s$. This means every element looks like

$a_{s - 1} X^{s - 1} + \dots + a_1 X + a_0$.

Since each of the $s$ coefficients can be any of the $p$ values from $\mathbb{F}_p$, the new field has exactly $p^s$ **elements**.

Arithmetic in this field works as follows:

- **Addition**: Standard polynomial addition, with coefficients added modulo $p$.
- **Multiplication**: Standard polynomial multiplication, followed by taking the **remainder** of the division by the irreducible polynomial $E(X)$.

---

### Example: Constructing $\mathbb{F}_{7^2} $

Let's work over our chosen field, $\mathbb{F}_7$. The polynomial $E(X) = X^2 + 3$ is irreducible over $\mathbb{F}_7$ (you can check it has no roots in $\mathbb{F}_7$). We can use it to construct the field $\mathbb{F}_{7^2}$, which has $7^2 = 49$ elements. The elements are all linear polynomials of the form $aX + b$ where $a, b \in \mathbb{F}_7$.

Let’s take two elements in this field: $A(X) = 2X + 5$ and $B(X) = 3X + 1$.

- **Addition**: $A(X) + B(X) = (2 + 3)X + (5 + 1) = 5X + 6$.

- **Multiplication**:  
  $A(X) \cdot B(X) = (2X + 5)(3X + 1) = 6X^2 + 17X + 5$.
  
  - First, reduce coefficients mod 7: $6X^2 + 3X + 5$.
  - Now, find the remainder when dividing by $E(X) = X^2 + 3$. From $X^2 + 3 = 0$, we know $X^2 \equiv -3 \equiv 4 \pmod{E(X)}, 7$.
  - Substitute this in: $6(4) + 3X + 5 = 24 + 3X + 5 = 29 + 3X$.
  - Finally, reduce the coefficients again: $1 + 3X$.

So, in $\mathbb{F}_{7^2}$,  
$(2X + 5) \cdot (3X + 1) = 3X + 1$.

In [1]:
import numpy as np
from numpy.polynomial import polynomial as P

def poly_add_mod(p1, p2, q):
    p_add = P.polyadd(p1.coef, p2.coef)
    return P.Polynomial(p_add % q)

def poly_mul_mod(p1, p2, q):
    p_mul = P.polymul(p1.coef, p2.coef)
    return P.Polynomial(p_mul % q)

def poly_divmod_mod(p1, p2, q):
    """
    Performs polynomial division over F_q.
    Returns quotient and remainder.
    """
    # Make copies to avoid modifying original objects
    p1_coef = np.copy(p1.coef).astype(int)
    p2_coef = np.copy(p2.coef).astype(int)

    if len(p1_coef) < len(p2_coef):
        return P.Polynomial([0]), p1

    # Find multiplicative inverse of leading coefficient of divisor
    lead_inv = pow(int(p2_coef[-1]), q - 2, q)
    
    quotient = np.zeros(len(p1_coef) - len(p2_coef) + 1, dtype=int)

    while len(p1_coef) >= len(p2_coef):
        deg_diff = len(p1_coef) - len(p2_coef)
        
        coef = (p1_coef[-1] * lead_inv) % q
        quotient[deg_diff] = coef

        term = (p2_coef * coef) % q
        
        # Ensure 'term' is aligned with the part of p1_coef it's being subtracted from
        sub_len = len(p1_coef) - deg_diff
        p1_coef[-sub_len:] = (p1_coef[-sub_len:] - term) % q
        
        # Remove leading zeros from the remainder
        while len(p1_coef) > 0 and p1_coef[-1] == 0:
            p1_coef = p1_coef[:-1]

    if len(p1_coef) == 0:
        remainder_poly = P.Polynomial([0])
    else:
        remainder_poly = P.Polynomial(p1_coef)
        
    return P.Polynomial(quotient), remainder_poly


# ---- Example in GF(7^2) ----
q = 7
# E(X) = X^2 + 3, which is irreducible over F_7
E = P.Polynomial([3, 0, 1]) 

A = P.Polynomial([5, 2]) # 2X + 5
B = P.Polynomial([1, 3]) # 3X + 1

print(f"Field: F_{q}")
print(f"Irreducible Polynomial E(X): {E}")
print("-" * 20)
print(f"A(X) = {A}")
print(f"B(X) = {B}")
print("-" * 20)


# Addition in GF(7^2)
C_add = poly_add_mod(A, B, q)
print(f"A(X) + B(X) = {C_add}")

# Multiplication in GF(7^2)
C_mul_prod = poly_mul_mod(A, B, q)
_, C_mul_rem = poly_divmod_mod(C_mul_prod, E, q)

print(f"A(X) * B(X) = {C_mul_prod}")
print(f"  ... mod E(X) => {C_mul_rem}")

Field: F_7
Irreducible Polynomial E(X): 3.0 + 0.0·x + 1.0·x²
--------------------
A(X) = 5.0 + 2.0·x
B(X) = 1.0 + 3.0·x
--------------------
A(X) + B(X) = 6.0 + 5.0·x
A(X) * B(X) = 5.0 + 3.0·x + 6.0·x²
  ... mod E(X) => 1.0 + 3.0·x


### Finding an Irreducible Polynomial

To construct these fields, we need a reliable way to find an irreducible polynomial of a given degree $s$. While there are deterministic ways, a simple and effective method is a **randomized algorithm**:

1. Generate a random **monic** polynomial $F(X)$ of degree $s$ with coefficients in $\mathbb{F}_q$.
2. Test if $F(X)$ is irreducible.
3. If it is, you're done! If not, go back to step 1.

The key is the test in step 2. A polynomial $F(X)$ of degree $s$ is irreducible over $\mathbb{F}_q$ **iff** it satisfies two conditions:

1. $F(X)$ divides $X^{q^{\,s}} - X$.
2. For every prime factor $d$ of $s$, the greatest common divisor
   $\gcd\!\big(F(X),\, X^{q^{\,s/d}} - X\big)$ is $1$.

### Condition 1: $F(X)$ must divide $X^{q^{s}} - X$

This is the **"Belonging" Test**. The polynomial $X^{q^{s}} - X$ is very special: its roots are **all the elements** of the field $\mathbb{F}_{q^s}$. If $F(X)$ is genuinely an irreducible polynomial of degree $s$, its roots must live in $\mathbb{F}_{q^s}$. Therefore, $F(X)$ must be a factor of the polynomial that defines the entire field. This check confirms that $F(X)$’s roots are in the correct target field.


### Condition 2: $\gcd(F(X),\, X^{q^{\,s/d}} - X) = 1$

This is the **"Minimality" Test**. The field $\mathbb{F}_{q^s}$ contains smaller subfields, like $\mathbb{F}_{q^{s/d}}$, for every prime factor $d$ of $s$. If the $\gcd$ in this test is not $1$, it means $F(X)$ shares roots with a polynomial that defines a smaller subfield. This would imply $F(X)$ is reducible, with factors belonging to that smaller field. An irreducible polynomial of degree $s$ must be “native” to $\mathbb{F}_{q^s}$, so this test ensures its roots aren’t secretly from a smaller subfield.

**In essence:** Condition 1 checks that the roots are in the **right field**, while Condition 2 checks they aren’t in any **smaller field**. Together, they prove that $F(X)$ must be irreducible of degree $s$.

This gives us a concrete algorithm to find our building blocks.

In [12]:

def get_prime_factors(n):
    factors = set()
    d = 2
    temp_n = n
    while d * d <= temp_n:
        if temp_n % d == 0:
            factors.add(d)
            while temp_n % d == 0:
                temp_n //= d
        d += 1
    if temp_n > 1:
        factors.add(temp_n)
    return factors

def poly_gcd_mod(p1, p2, q):
    a, b = p1, p2
    while b.degree() > -1 and np.any(b.coef != 0): 
        _, r = poly_divmod_mod(a, b, q)
        a, b = b, r
    # Normalize to make it monic
    if a.degree() > -1:
        lead_inv = pow(int(a.coef[-1]), q - 2, q)
        a = P.Polynomial((a.coef * lead_inv) % q)
    return a

def poly_pow_mod(base, exp, mod_poly, q):
    res = P.Polynomial([1])
    base_rem = poly_divmod_mod(base, mod_poly, q)[1]

    while exp > 0:
        if exp % 2 == 1:
            res_prod = poly_mul_mod(res, base_rem, q)
            res = poly_divmod_mod(res_prod, mod_poly, q)[1]
        
        base_rem_sq = poly_mul_mod(base_rem, base_rem, q)
        base_rem = poly_divmod_mod(base_rem_sq, mod_poly, q)[1]
        exp //= 2
    return res

def is_irreducible(F, q, s):
    """
    Tests if a polynomial F of degree s is irreducible over F_q.
    """
    # 1. Test if F(X) divides X^(q^s) - X
    # This is equivalent to X^(q^s) = X (mod F(X))
    X = P.Polynomial([0, 1])
    # Use modular exponentiation for efficiency
    x_pow = poly_pow_mod(X, q**s, F, q)
    
    rem = poly_divmod_mod(x_pow - X, F, q)[1]
    if rem.degree() > -1 and np.any(rem.coef != 0):
        return False # F(X) does not divide X^(q^s) - X

    # 2. Test gcd condition for all prime factors of s
    prime_factors_s = get_prime_factors(s)
    for d in prime_factors_s:
        exp = q**(s // d)
        x_pow = poly_pow_mod(X, exp, F, q)
        gcd = poly_gcd_mod(F, x_pow - X, q)
        if gcd.degree() > 0:
            return False # gcd is not 1

    return True

def find_irreducible_poly(q, s):
    """Finds a random monic irreducible polynomial of degree s over F_q."""
    print(f"\nSearching for a monic irreducible polynomial of degree {s} over F_{q}...")
    while True:
        # Generate a random monic polynomial of degree s
        coeffs = np.random.randint(0, q, s)
        coeffs = np.append(coeffs, 1) # Make it monic
        F = P.Polynomial(coeffs)
        
        if is_irreducible(F, q, s):
            print(f"Found one: {F}")
            return F

# --- Example of finding a polynomial ---
find_irreducible_poly(q=7, s=2)
find_irreducible_poly(q=2, s=4);


Searching for a monic irreducible polynomial of degree 2 over F_7...
Found one: 5.0 + 3.0·x + 1.0·x²

Searching for a monic irreducible polynomial of degree 4 over F_2...
Found one: 1.0 + 0.0·x + 0.0·x² + 1.0·x³ + 1.0·x⁴


## Part 2: Constructing Reed–Solomon Codes

Now we get to the core of the topic. A Reed-Solomon (RS) code is created through a beautifully simple process: messages
are turned into polynomials, and codewords are generated by evaluating those polynomials at several points.

### How Reed–Solomon Encoding Works

The definition of an RS code gives us a clear recipe for encoding:

1. **Start with the parameters:**
   - A finite field, $\mathbb{F}_q$. We'll stick with $\mathbb{F}_7$.
   - A **message length** $k$.
   - A **block length** (codeword length) $n$. We must have $k \le n$.

2. **Map Message to Polynomial:**  
   Take a message $\mathbf{m} = (m_0, m_1, \dots, m_{k-1})$, which is a vector of $k$ symbols from $\mathbb{F}_q$.  
   We treat these symbols as coefficients to form a **message polynomial** of degree at most $k-1$:
   $$
   f_{\mathbf{m}}(X) = m_0 + m_1 X + \cdots + m_{k-1} X^{k-1}.
   $$

3. **Evaluate:**  
   Choose $n$ distinct **evaluation points** $(\alpha_1,\alpha_2,\dots,\alpha_n)$ from $\mathbb{F}_q$.  
   The final codeword is the evaluation of the message polynomial at each of these points:
   $$
   \text{Codeword}=(f_{\mathbf{m}}(\alpha_1),\, f_{\mathbf{m}}(\alpha_2),\, \dots,\, f_{\mathbf{m}}(\alpha_n)).
   $$

---

### Example: An RS code over $\mathbb{F}_7$

Define an RS code with parameters $[n,k]=[5,3]$ over $\mathbb{F}_7$.

- **Message:** A vector of $k=3$ symbols, e.g., $\mathbf{m}=(6,1,2)$.
- **Polynomial:** This message maps to the polynomial $f(X)=2X^2 + X + 6$.
- **Evaluation Points:** We need $n=5$ distinct points from $\mathbb{F}_7$. Let's choose the set $\{1,2,3,4,5\}$.
- **Encoding:** Calculate $f(1), f(2), f(3), f(4), f(5)$ modulo $7$.
  - $f(1)=2(1)^2+1+6=9 \equiv 2 \pmod{7}$  
  - $f(2)=2(2)^2+2+6=16 \equiv 2 \pmod{7}$  
  - …and so on.

The resulting 5-symbol vector is our codeword. *(For the example above, it is $(2,2,6,0,5)$.)*

In [25]:
import numpy as np
from numpy.polynomial import polynomial as P

# --- RS Encoder ---

def rs_encoder(message_vec, n, q, eval_points):
    """
    Encodes a message vector into a Reed-Solomon codeword.
    
    Args:
        message_vec (list or np.array): The k-element message from F_q.
        n (int): The block length of the code.
        q (int): The size of the finite field.
        eval_points (list or np.array): The n distinct evaluation points.
        
    Returns:
        np.array: The n-element codeword.
    """
    if len(eval_points) != n:
        raise ValueError("The number of evaluation points must be equal to n.")
    
    message_poly = P.Polynomial(message_vec)
    
    print(f"Message m = {message_vec}  -->  Polynomial f(X) = {message_poly}")
    
    # Evaluate the polynomial at each point in the evaluation set
    codeword = []
    for alpha in eval_points:
        val = message_poly(alpha) % q
        codeword.append(int(val))
        
    return np.array(codeword)

# --- Example of Encoding ---
q = 7
n = 5
k = 3
# Choose n distinct evaluation points from F_7
eval_points = [1, 2, 3, 4, 5] 

# A message m is a vector of k=3 elements from F_7
m1 = [6, 1, 2] # Represents f(X) = 6 + 1*X + 2*X^2

print(f"--- Encoding with RS[{n}, {k}] over F_{q} ---")
c1 = rs_encoder(m1, n, q, eval_points)
print(f"Evaluation Points: {eval_points}")
print(f"Resulting Codeword c1 = {c1}")

--- Encoding with RS[5, 3] over F_7 ---
Message m = [6, 1, 2]  -->  Polynomial f(X) = 6.0 + 1.0·x + 2.0·x²
Evaluation Points: [1, 2, 3, 4, 5]
Resulting Codeword c1 = [2 2 6 0 5]


## Properties of Reed–Solomon Codes

RS codes are widely used because they have excellent, provable
properties.

- **Linearity:** RS codes are linear codes. This means
  that if you add two messages and then encode the sum, you get the
  same result as if you first encode each message and then add the
  resulting codewords:
  $$
  \mathrm{Encode}(\mathbf m_1 + \mathbf m_2)
  \;=\;
  \mathrm{Encode}(\mathbf m_1) + \mathrm{Encode}(\mathbf m_2).
  $$

- **Distance:** The minimum distance
  of an RS code is
  $$
  d = n - k + 1.
  $$
  This is the largest possible distance for any linear code with
  parameters $n$ and $k$, a limit known as the **Singleton Bound**.
  Because RS codes achieve this bound, they are considered
  **optimal** in terms of their distance and are known as
  **Maximum Distance Separable (MDS)** codes.

  The proof is elegant: consider two different messages
  $\mathbf m_1$ and $\mathbf m_2$, and their polynomials
  $f_1(X)$ and $f_2(X)$. The number of places their codewords
  agree is the number of roots of the difference polynomial
  $$
  g(X) = f_1(X) - f_2(X).
  $$
  Since the degree of $g(X)$ is at most $k-1$, it can have at most
  $k-1$ roots. Therefore, the codewords can agree in at most
  $k-1$ positions, meaning they must differ in at least
  $$
  n - (k - 1) = n - k + 1
  $$
  positions.

Let's verify these properties with our code.

In [41]:
def hamming_distance(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("Vectors must have the same length.")
    return np.sum(v1 != v2)

# --- Verifying Properties ---

# 1. Linearity
print("\n--- Verifying Linearity ---")
m2 = [1, 3, 1] 
print("Recalling our initial messages and codewords:")
print(f"m1 = {m1}")
c1 = rs_encoder(m1, n, q, eval_points) 
print(f"--> c1 = {c1}\n")

print(f"m2 = {m2}")
c2 = rs_encoder(m2, n, q, eval_points)
print(f"--> c2 = {c2}\n")


# Encode(m1 + m2)
m_sum_np = (np.array(m1) + np.array(m2)) % q
m_sum = m_sum_np.tolist() 

print("---")
print("Encoding the sum of messages m1+m2:")
c_sum = rs_encoder(m_sum, n, q, eval_points)
print(f"Result: {c_sum}\n")

# Encode(m1) + Encode(m2)
c1_plus_c2 = (c1 + c2) % q
print("Adding the codewords c1+c2:")
print(f"Result: {c1_plus_c2}\n")

assert np.array_equal(c_sum, c1_plus_c2), "Linearity test failed!"
print("Linearity holds: Encode(m1+m2) == Encode(m1) + Encode(m2)")


# 2. Distance
print("\n--- Verifying Distance ---")
d_theory = n - k + 1
print(f"Theoretical minimum distance d = n-k+1 = {n}-{k}+1 = {d_theory}")

dist_c1_c2 = hamming_distance(c1, c2)
print(f"\nActual distance between c1 and c2: {dist_c1_c2}")
assert dist_c1_c2 >= d_theory

# Let's try another random message
m3 = [5, 4, 1]
print("\nComparing c1 with a new codeword c3:\n")
c3 = rs_encoder(m3, n, q, eval_points)
print(f"--> c3 = {c3}")
dist_c1_c3 = hamming_distance(c1, c3)
print(f"Actual distance between c1 and c3: {dist_c1_c3}")
assert dist_c1_c3 >= d_theory

print("\nConstructing a pair with the minimum possible distance.")

c_zero = rs_encoder(m_zero, n, q, eval_points)
print(f"\n--> Codeword for all-zero message: {c_zero}")
c_diff = rs_encoder(m_diff, n, q, eval_points)
print(f"\n--> Codeword for difference message: {c_diff}")

dist_min = hamming_distance(c_zero, c_diff)
print(f"\nThe distance is {dist_min}. This matches the theoretical minimum d_min = {d_theory}.")
assert dist_min == d_theory


print("\nDistance property holds and we have demonstrated the minimum case.")



--- Verifying Linearity ---
Recalling our initial messages and codewords:
m1 = [6, 1, 2]
Message m = [6, 1, 2]  -->  Polynomial f(X) = 6.0 + 1.0·x + 2.0·x²
--> c1 = [2 2 6 0 5]

m2 = [1, 3, 1]
Message m = [1, 3, 1]  -->  Polynomial f(X) = 1.0 + 3.0·x + 1.0·x²
--> c2 = [5 4 5 1 6]

---
Encoding the sum of messages m1+m2:
Message m = [0, 4, 3]  -->  Polynomial f(X) = 0.0 + 4.0·x + 3.0·x²
Result: [0 6 4 1 4]

Adding the codewords c1+c2:
Result: [0 6 4 1 4]

Linearity holds: Encode(m1+m2) == Encode(m1) + Encode(m2)

--- Verifying Distance ---
Theoretical minimum distance d = n-k+1 = 5-3+1 = 3

Actual distance between c1 and c2: 5

Comparing c1 with a new codeword c3:

Message m = [5, 4, 1]  -->  Polynomial f(X) = 5.0 + 4.0·x + 1.0·x²
--> c3 = [3 3 5 2 1]
Actual distance between c1 and c3: 5

Constructing a pair with the minimum possible distance.
Message m = [0, 0, 0]  -->  Polynomial f(X) = 0.0 + 0.0·x + 0.0·x²

--> Codeword for all-zero message: [0 0 0 0 0]
Message m = [2, 4, 1]  -->  P

### The Missing Link: Why Do RS Codes Need Field Extensions?

So far, we've introduced two major concepts:

1. **Field Extensions** $F_{p^s}$: A way to construct large fields with $q = p^s$ elements.
2. **Reed–Solomon Codes**: A method of encoding that works over any finite field $F_q$.

But why are they so important to *each other*? The connection lies in the **length of the codeword**.

#### The Problem: The Alphabet Size Constraint

The Reed–Solomon encoding process requires us to choose $n$ distinct evaluation points from our field $F_q$. This leads to a fundamental constraint: the block length $n$ must be less than or equal to the field size $q$ ($n \le q$).

If we only use simple prime fields, our codes are forced to be very short. For example, in our notebook we used $F_7$. The longest possible RS code we could build over this field is just **7 symbols** long. This is far too short for most real-world applications like data storage or digital communication, where we need to handle thousands of bits of data.

#### The Solution: Creating Large Alphabets

This is where **field extensions** become essential. They are the tool we use to construct massive alphabets from small, simple ones.

The most common choice in practice is to start with the simplest field, $F_2 = \{0,1\}$, and build an extension. For example, by choosing an irreducible polynomial of degree $8$ over $F_2$, we can construct the field extension $F_{2^8}$.

This new field has $q = 2^8 = 256$ elements. Now, our alphabet size is $256$. This means we can create RS codes with a block length up to $n = 255$ (using the non-zero elements as evaluation points).

Crucially, each “symbol” in $F_{2^8}$ corresponds to a block of $8$ bits, which is exactly **one byte**. So, we can think of an RS code over $F_{2^8}$ as a code that operates on **bytes** instead of individual bits.

The real-world example is the **RS[255, 223]** code over $F_{2^8}$:

- It takes a message of $k = 223$ bytes.
- It adds $n - k = 32$ parity bytes.
- It produces a codeword of $n = 255$ bytes.
- This code can correct up to $(n - k)/2 = 16$ byte errors anywhere in the codeword.

### Part 3: The MDS Property  

We've seen that Reed–Solomon codes are optimal with respect to their distance, meeting the Singleton Bound. This class of codes has a special name.

> **MDS Codes:** An $[n, k, d]_q$ code is called **Maximum Distance Separable (MDS)** if its parameters satisfy $d = n - k + 1$.

As we proved, Reed–Solomon codes are MDS codes. This property implies something profound about their structure, which makes them appear very "random" and uniform.

### The Core Property of MDS Codes

The key property of MDS codes (stated in Proposition 5.3.3) is a powerful statement about their uniformity. Let's use our running example of an RS$[5, 3]$ code over the field $\mathbb{F}_7$ to understand it.

For this code, we have:

- $q = 7$ (our alphabet is $\{0, 1, 2, 3, 4, 5, 6\}$)
- $k = 3$ (our messages are 3 symbols long)
- $n = 5$ (our codewords are 5 symbols long)

There are $q^k = 7^3 = 343$ possible messages we can encode, ranging from $(0, 0, 0), (0, 0, 1),$ all the way to $(6, 6, 6)$. Each of these 343 messages is encoded into a unique codeword of length 5.

Imagine a table with 343 rows and 5 columns. Each row is a different codeword.

| Message   | Codeword (5 columns)                      |
|-----------|--------------------------------------------|
| (0,0,0)   | $(c_{1,1}, c_{1,2}, c_{1,3}, c_{1,4}, c_{1,5})$ |
| (0,0,1)   | $(c_{2,1}, c_{2,2}, c_{2,3}, c_{2,4}, c_{2,5})$ |
| ...       | ...                                        |
| (6,6,6)   | $(c_{343,1}, c_{343,2}, c_{343,3}, c_{343,4}, c_{343,5})$ |

The MDS property guarantees the following: if you pick **any** $k = 3$ columns from this table (say, columns 2, 3, and 5) and look down the 343 rows, the list of 3-symbol vectors you see will be a perfect list of **all 343 possible combinations**, from $(0,0,0)$ to $(6,6,6)$, with each appearing exactly once.

This remarkable uniformity is what makes MDS codes so special. It means no information is "clumped together"; it is spread out as evenly as possible across the codeword.

### How is this possible?

This property is a direct result of the code’s polynomial structure. It guarantees that for **any** choice of $k$ positions and any desired $k$-symbol target vector $\mathbf{v}$, we can find a unique message polynomial $f(X)$ that “hits” those values. This requires solving a system of linear equations where the matrix is an invertible **Vandermonde matrix**, which always has a unique solution.

### Demonstrating the MDS Property

Let's prove this with code. We will take our RS$[5, 3]_7$ code and challenge it: can we find a message that produces the specific sub-vector `[4, 0, 1]` at positions `0, 2, 4`?

This means we need to find a message  
$\textbf{m} = (m_0, m_1, m_2)$  
such that its polynomial $f(X)$ satisfies:

- $f(\alpha_0) = f(1) = 4$
- $f(\alpha_2) = f(3) = 0$
- $f(\alpha_4) = f(5) = 1$

In [48]:
import numpy as np
from numpy.polynomial import polynomial as P

# --- Helper function for modular inverse of a matrix ---
def matrix_inverse_mod(matrix, q):
    """Calculates the inverse of a matrix in a finite field F_q."""
    det = int(np.round(np.linalg.det(matrix)))
    det_inv = pow(det, q - 2, q)
    adjugate = np.round(det * np.linalg.inv(matrix)).astype(int)
    return (adjugate * det_inv) % q

def find_message_for_projection(target_indices, target_vector, eval_points, q, k):
    """
    Finds the unique message that produces a target vector at specific indices.
    Also returns the Vandermonde matrix used.
    """
    sub_eval_points = [eval_points[i] for i in target_indices]
    
    # Construct the k x k Vandermonde matrix
    V = np.vander(sub_eval_points, k, increasing=True).astype(int)
    
    # Find the inverse of the Vandermonde matrix mod q
    V_inv = matrix_inverse_mod(V, q)
    
    # Solve for the message m = V_inv * v
    message_np = (V_inv @ np.array(target_vector)) % q
    message = message_np.tolist()
    
    return message, V

# --- Setup from before ---
q = 7
n = 5
k = 3
eval_points = [1, 2, 3, 4, 5]

# --- The Challenge ---
target_indices = [0, 2, 4] 
target_vector = [4, 0, 1]

print("Challenge: Find `m` for a specific projection.")
print(f"Target vector v = {target_vector} at indices = {target_indices}")

found_message, V_matrix = find_message_for_projection(target_indices, target_vector, eval_points, q, k)

# --- Show the equation V * m = v with numbers ---
print("\n--- Verifying the equation V * m = v ---\n")
print("V (Vandermonde matrix):")
print(V_matrix)
print(f"\nm (Found message): {found_message}")
print(f"\nv (Target vector): {target_vector}")

# Check the multiplication
result_vector = (V_matrix @ np.array(found_message)) % q
print(f"\nResult of V * m (mod {q}): {result_vector.tolist()}")
assert np.array_equal(result_vector, target_vector)


# --- Final verification by encoding ---
print("\n--- Verifying by encoding the found message ---")
found_codeword = rs_encoder(found_message, n, q, eval_points)
print(f"Full codeword c = {found_codeword}")

# Check if the codeword matches the target at the specified indices
projection = [int(found_codeword[i]) for i in target_indices] # Fix for np.int64
print(f"Codeword projection at indices {target_indices}: {projection}")

assert projection == target_vector
print("\n Success! The property holds. A unique message was found.")

Challenge: Find `m` for a specific projection.
Target vector v = [4, 0, 1] at indices = [0, 2, 4]

--- Verifying the equation V * m = v ---

V (Vandermonde matrix):
[[ 1  1  1]
 [ 1  3  9]
 [ 1  5 25]]

m (Found message): [0, 6, 5]

v (Target vector): [4, 0, 1]

Result of V * m (mod 7): [4, 0, 1]

--- Verifying by encoding the found message ---
Message m = [0, 6, 5]  -->  Polynomial f(X) = 0.0 + 6.0·x + 5.0·x²
Full codeword c = [4 4 0 6 1]
Codeword projection at indices [0, 2, 4]: [4, 0, 1]

 Success! The property holds. A unique message was found.
